In [18]:
import hashlib
import sys
import os

In [19]:
# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [20]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [21]:
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

In [22]:
# Colunas da dimensão CURSO
colunas_curso = [
    "codCursoIngresso",
    "cursoIngressoUFRJ",
    "codCursoAtual",
    "cursoAtual",
]

# Seleção e remoção de duplicatas
d_curso = df_silver[colunas_curso].drop_duplicates().reset_index(drop=True)

In [23]:
# Criação do sk_d_curso usando SHA-256
d_curso["SK_D_CURSO"] = d_curso.apply(
    lambda x: hashlib.sha256(
        str(x["codCursoIngresso"] + x["codCursoAtual"]).encode()
    ).hexdigest(),
    axis=1,
)

# Reordenação das colunas
d_curso = d_curso[["SK_D_CURSO"] + colunas_curso]

# Padronizando nome das colunas
d_curso = d_curso.rename(
    columns={
        "codCursoIngresso": "CD_CURSO_INGRESSO",
        "cursoIngressoUFRJ": "DS_NOME_CURSO_INGRESSO",
        "codCursoAtual": "CD_CURSO_ATUAL",
        "cursoAtual": "DS_NOME_CURSO_ATUAL",
    }
)

In [24]:
# Salvar dados na camada Gold
carregar_dados_database(d_curso, "d_curso.parquet", "gold")